In [ ]:
from torch import nn
from torch_mist.estimators import TransformedMIEstimator

transform = nn.Sequential(
            nn.Linear(n_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 2)
        )
estimator = TransformedMIEstimator(
    base_estimator=nwj(
        x_dim=2,
        y_dim=2,
        hidden_dims=[128],
        neg_samples=1
    ),
    transforms={
        'x': transform,
        'y': transform,
    }
)

train_log = train_mi_estimator(
    estimator,
    x=samples['x'],
    y=samples['y'],
    **train_parameters
)

grid = sns.FacetGrid(train_log, col='name')

grid.map(sns.lineplot,  'epoch', 'value', errorbar='sd')
plt.axhline(y=true_mi, ls='--', color='k', label='True $I(x;y)$')
plt.xlim(0, 20)
plt.legend()




z = transform(samples['y']).data
Q_y = kmeans_quantization(z, n_bins=32)


sns.scatterplot(x=z[:,0], y=z[:,1], hue=Q_y(z), marker='.', alpha=0.05, legend=False)


from torch_mist.estimators import TransformedMIEstimator

z_dim = 8
transform = nn.Sequential(
            nn.Linear(n_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, z_dim)
        )

discriminative_estimator = mine(
    x_dim=z_dim,
    y_dim=z_dim,
    neg_samples=1,
    hidden_dims=[256, 128]
)

hybrid_estimator = TransformedMIEstimator(
        base_estimator=discriminative_estimator,
        transforms={
            'x': transform,
            'y': transform,
        }
    )

train_log = train_mi_estimator(
        hybrid_estimator,
        x=samples['x'],
        y=samples['y'],
        **train_parameters
    )
log['step'] = 0
log = pd.concat([log, train_log])

train_parameters['max_epochs'] = 20
train_parameters['optimizer_params']={'lr':5e-4}

for i in range(1,7):
    old_transform = freeze(deepcopy(transform))
    z = transform(samples['y']).data
    Q_y = kmeans_quantization(z, n_bins=min(2**i,64))
    
    hybrid_estimator = TransformedMIEstimator(
        base_estimator=PQHybridMIEstimator(
            pq_estimator=pq(Q_y=Q_y, x_dim=z_dim, hidden_dims=[128]),
            discriminative_estimator=discriminative_estimator,
        ),
        transforms={
            'x': transform,
            'y': old_transform,
        }
    )

    train_log = train_mi_estimator(
        hybrid_estimator,
        x=samples['x'],
        y=samples['y'],
        **train_parameters
    )
    train_log['step'] = i
    log = pd.concat([log, train_log])


grid = sns.FacetGrid(log, col='name', hue='step', sharey=False)

grid.map(sns.lineplot,  'epoch', 'value', errorbar='sd')
plt.axhline(y=true_mi, ls='--', color='k', label='True $I(x;y)$')
plt.xlim(0, 20)
grid.add_legend()

z_x = hybrid_estimator.transforms['x'](samples['x']).data
z_y = hybrid_estimator.transforms['y'](samples['y']).data
f, ax = plt.subplots(1,2, figsize=(12,5))
dis_y = torch.argmax(hybrid_estimator.base_estimator.generative_estimator.base_estimator.q_Y_given_X.condition(z_x).logits,-1)

sns.scatterplot(x=z_x[:,0], y=z_x[:,1], hue=dis_y, marker='.', alpha=0.05, legend=False, ax=ax[0])
sns.scatterplot(x=z_y[:,0], y=z_y[:,1], hue=Q_y(z_y), marker='.', alpha=0.05, legend=False, ax=ax[1])


aa
$$
I(x;y)\le \mathbb{E}_{p(x,y)}\left[F_\theta(x,y)+\log\frac{r_\theta(y|x)}{p(y)}\right] - \mathbb{E}_{p(x)}\log \mathbb{E}_{r_\theta(y|x)}\left[e^{F_\theta(x,y)+\log\frac{r_\theta(y|x)}{p(y)}}\right].
$$
Defining
$$
g_\theta(x,y) = F_\theta(x,y)+\log\frac{r_\theta(y|x)}{p(y)}
$$
we have
$$
I(x;y) \le \mathbb{E}_{p(x,y)}\left[g_\theta(x,y)\right] - \mathbb{E}_{p(x)}\log \mathbb{E}_{p(y)}\left[e^{g_\theta(x,y)}\right].
$$
$$
\frac{p(x)p(y|Q(x))}{p(x)p(y)} = \frac{p(y|Q(x))}{p(y)} =  \frac{p(x,y|Q(x))}{p(y)p(x|y,Q(x))} = \frac{p(x|Q(x))p(y|x)}{p(y)p(x|y,Q(x))} = \frac{p(x|Q(x))p(x|y)}{p(x)p(x|y,Q(x))} 
$$

$$
\frac{p(x|Q(x))p(x|y)}{p(x)p(x|y,Q(x))} = \frac{p(x|Q(x))p(Q(x)|y)}{p(x)p(Q(x)|x)}
$$
